In [ ]:
from qiskit import QuantumCircuit, transpile, ClassicalRegister, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
import numpy as np

sim = AerSimulator()

## Q1: BB84 Quantum Key Distribution

In [ ]:
np.random.seed(42)
n_bits = 100

alice_bases = np.random.randint(0, 2, n_bits)
alice_key = np.random.randint(0, 2, n_bits)
bob_bases = np.random.randint(0, 2, n_bits)

print("Alice bases (first 20):", alice_bases[:20])
print("Alice key (first 20):  ", alice_key[:20])
print("Bob bases (first 20):  ", bob_bases[:20])

In [ ]:
def alice_encode(bit, basis):
    qc = QuantumCircuit(1, 1)
    if bit == 1:
        qc.x(0)
    if basis == 1:
        qc.h(0)
    return qc

print("Alice encoding examples:")
for bit in [0, 1]:
    for basis in [0, 1]:
        basis_name = 'X' if basis == 1 else 'Z'
        qc = alice_encode(bit, basis)
        sv = Statevector.from_instruction(qc)
        print(f"  bit={bit}, basis={basis_name}: {sv}")

In [ ]:
def bob_measure(qc, basis):
    if basis == 1:
        qc.h(0)
    qc.measure(0, 0)
    return qc

bob_results = []
for i in range(n_bits):
    qc = alice_encode(alice_key[i], alice_bases[i])
    qc = bob_measure(qc, bob_bases[i])
    tqc = transpile(qc, sim)
    result = sim.run(tqc, shots=1).result()
    measured = int(list(result.get_counts().keys())[0])
    bob_results.append(measured)

bob_results = np.array(bob_results)
print("Bob results (first 20):", bob_results[:20])

In [ ]:
matching_indices = np.where(alice_bases == bob_bases)[0]
sifted_key = bob_results[matching_indices]

print(f"Matching basis positions: {len(matching_indices)} out of {n_bits}")
print(f"Sifted key length: {len(sifted_key)}")
print(f"Sifted key: {sifted_key}")

alice_sifted = alice_key[matching_indices]
match_rate = np.sum(sifted_key == alice_sifted) / len(sifted_key) * 100
print(f"Key match rate: {match_rate:.1f}%")

In [ ]:
encrypted_message = "01001000 01100101 01101100 01101100 01101111"

encrypted_bits = encrypted_message.replace(" ", "")
key_repeated = np.tile(sifted_key, (len(encrypted_bits) // len(sifted_key)) + 1)[:len(encrypted_bits)]

decrypted_bits = ''.join([str(int(encrypted_bits[i]) ^ key_repeated[i]) for i in range(len(encrypted_bits))])

decrypted_chars = [chr(int(decrypted_bits[i:i+8], 2)) for i in range(0, len(decrypted_bits), 8)]
decrypted_message = ''.join(decrypted_chars)

print(f"Encrypted (binary): {encrypted_message}")
print(f"Key used: {key_repeated[:len(encrypted_bits)]}")
print(f"Decrypted message: {decrypted_message}")

## Q2: Quantum Teleportation

In [ ]:
alpha = np.cos(np.pi/8)
beta = np.sin(np.pi/8)
print(f"Initial state to teleport: {alpha:.4f}|0> + {beta:.4f}|1>")
print(f"  alpha = cos(pi/8) = {alpha:.4f}")
print(f"  beta = sin(pi/8) = {beta:.4f}")

qr = QuantumRegister(3, 'q')
crz = ClassicalRegister(1, 'crz')
crx = ClassicalRegister(1, 'crx')
qc = QuantumCircuit(qr, crz, crx)

qc.ry(2 * np.arctan2(beta, alpha), 0)
qc.barrier(label='State')

qc.h(1)
qc.cx(1, 2)
qc.barrier(label='Bell')

qc.cx(0, 1)
qc.h(0)
qc.barrier(label='Meas')

qc.measure(0, crz)
qc.measure(1, crx)

qc.x(2).c_if(crx, 1)
qc.z(2).c_if(crz, 1)

display(qc.draw('mpl'))

In [ ]:
print("Teleportation Protocol:")
print("  q0: State to teleport (Alice's qubit)")
print("  q1, q2: Bell pair |Phi+> = (|00> + |11>)/sqrt(2)")
print("    q1 with Alice, q2 with Bob")
print("")
print("Post-processing on q2:")
print("  crx=1 -> Apply X gate")
print("  crz=1 -> Apply Z gate")

tqc = transpile(qc, sim)
result = sim.run(tqc, shots=4096).result()
counts = result.get_counts()
print(f"\nMeasurement outcomes: {counts}")

In [ ]:
sv_sim = AerSimulator(method='statevector')

print(f"Verification (target: {alpha:.4f}|0> + {beta:.4f}|1>):")
for m0 in [0, 1]:
    for m1 in [0, 1]:
        qc_test = QuantumCircuit(3)
        qc_test.ry(2 * np.arctan2(beta, alpha), 0)
        qc_test.h(1)
        qc_test.cx(1, 2)
        qc_test.cx(0, 1)
        qc_test.h(0)
        
        if m0 == 1:
            qc_test.z(0)
        if m1 == 1:
            qc_test.z(1)
        
        qc_test.reset(0)
        qc_test.reset(1)
        if m0 == 1:
            qc_test.x(0)
        if m1 == 1:
            qc_test.x(1)
        
        if m1 == 1:
            qc_test.x(2)
        if m0 == 1:
            qc_test.z(2)
        
        qc_test.save_statevector()
        result = sv_sim.run(qc_test).result()
        sv = result.get_statevector()
        
        amp_0 = sv[0]
        amp_1 = sv[4]
        
        print(f"  Measurement ({m0},{m1}): q2 = {np.real(amp_0):.4f}|0> + {np.real(amp_1):.4f}|1>")

## Q3: GHZ-like State Teleportation

In [ ]:
def create_ghz_like_state():
    qc = QuantumCircuit(3)
    qc.h(0)
    qc.h(1)
    qc.cx(0, 2)
    qc.cx(1, 2)
    return qc

qc_ghz = create_ghz_like_state()
sv = Statevector.from_instruction(qc_ghz)
print("GHZ-like state: (|001> + |010> + |100> + |111>)/2")
print(f"Statevector: {sv}")
display(qc_ghz.draw('mpl'))

In [ ]:
alpha = np.cos(np.pi/6)
beta = np.sin(np.pi/6)
print(f"State to teleport: {alpha:.4f}|0> + {beta:.4f}|1>")

qr = QuantumRegister(4, 'q')
cr_bell = ClassicalRegister(2, 'bell')
cr_bob = ClassicalRegister(1, 'bob')
qc = QuantumCircuit(qr, cr_bell, cr_bob)

qc.ry(2 * np.arctan2(beta, alpha), 0)
qc.barrier(label='psi')

qc.h(1)
qc.h(2)
qc.cx(1, 3)
qc.cx(2, 3)
qc.barrier(label='GHZ')

qc.cx(0, 1)
qc.h(0)
qc.measure(0, cr_bell[0])
qc.measure(1, cr_bell[1])
qc.barrier(label='Alice')

qc.measure(2, cr_bob[0])
qc.barrier(label='Bob')

display(qc.draw('mpl'))

In [ ]:
print("Correction table (Bell measurement, Bob Z-measurement -> Correction on Charlie):")
print("  (00, 0) -> I")
print("  (00, 1) -> Z")
print("  (01, 0) -> X")
print("  (01, 1) -> XZ")
print("  (10, 0) -> Z")
print("  (10, 1) -> I")
print("  (11, 0) -> XZ")
print("  (11, 1) -> X")

sv_sim = AerSimulator(method='statevector')

print(f"\nVerification (target: {alpha:.4f}|0> + {beta:.4f}|1>):")
for bell in range(4):
    for bob in range(2):
        qc_test = QuantumCircuit(4)
        qc_test.ry(2 * np.arctan2(beta, alpha), 0)
        
        qc_test.h(1)
        qc_test.h(2)
        qc_test.cx(1, 3)
        qc_test.cx(2, 3)
        
        qc_test.cx(0, 1)
        qc_test.h(0)
        
        for i in range(2):
            qc_test.reset(i)
            if (bell >> i) & 1:
                qc_test.x(i)
        
        qc_test.reset(2)
        if bob == 1:
            qc_test.x(2)
        
        if bell in [1, 3]:
            qc_test.x(3)
        if (bell in [0, 1] and bob == 1) or (bell in [2, 3] and bob == 0):
            qc_test.z(3)
        
        qc_test.save_statevector()
        result = sv_sim.run(qc_test).result()
        sv = result.get_statevector()
        
        q3_0 = sv[0]
        q3_1 = sv[8]
        
        print(f"  Bell={bell:02b}, Bob={bob}: Charlie = {np.real(q3_0):.4f}|0> + {np.real(q3_1):.4f}|1>")

In [ ]:
qr = QuantumRegister(4, 'q')
cr_bell = ClassicalRegister(2, 'bell')
cr_bob = ClassicalRegister(1, 'bob')
qc_full = QuantumCircuit(qr, cr_bell, cr_bob)

qc_full.ry(2 * np.arctan2(beta, alpha), 0)
qc_full.barrier()

qc_full.h(1)
qc_full.h(2)
qc_full.cx(1, 3)
qc_full.cx(2, 3)
qc_full.barrier()

qc_full.cx(0, 1)
qc_full.h(0)
qc_full.measure(0, cr_bell[0])
qc_full.measure(1, cr_bell[1])
qc_full.barrier()

qc_full.measure(2, cr_bob[0])
qc_full.barrier()

with qc_full.if_test((cr_bell, 1)):
    qc_full.x(3)
with qc_full.if_test((cr_bell, 3)):
    qc_full.x(3)

display(qc_full.draw('mpl'))

print("\nFull circuit with conditional corrections on Charlie's qubit (q3)")
print("Qubits: q0=psi (Alice), q1=Alice's GHZ, q2=Bob (controller), q3=Charlie (receiver)")